# Repeat from colab jupyter

In [2]:
# Dockerfile has state installed
# Looking at your Dockerfile, the issue is that uv tool install creates an isolated environment that may not persist properly in the Docker container
# cause wandb config files missing 
! state --help

Traceback (most recent call last):
  File "/usr/local/bin/state", line 5, in <module>
    from state.__main__ import main
ModuleNotFoundError: No module named 'state'


In [4]:
# install the package
! uv run state --help

Traceback (most recent call last):
  File "/usr/local/bin/state", line 5, in <module>
    from state.__main__ import main
ModuleNotFoundError: No module named 'state'


# Clone the repository

In [3]:
# Clone the repository
! git clone https://github.com/ArcInstitute/state.git
%cd state

# Colab-specific config for pytorch lightning
import os
os.environ['MPLBACKEND'] = 'Agg'

Cloning into 'state'...
remote: Enumerating objects: 6345, done.
remote: Counting objects: 100% (505/505), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 6345 (delta 367), reused 345 (delta 274), pack-reused 5840 (from 2)
Receiving objects: 100% (6345/6345), 122.69 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (3866/3866), done.
/workspace/state


# Download the Replogle-Nadig training dataset.

In [3]:
%pip install requests tqdm

import requests
from tqdm.auto import tqdm  # picks the best bar for the environment

url = "https://storage.googleapis.com/vcc_data_prod/datasets/state/competition_support_set.zip"
output_path = "competition_support_set.zip"

# stream the download so we can track progress
response = requests.get(url, stream=True)
total = int(response.headers.get("content-length", 0))

with open(output_path, "wb") as f, tqdm(
    total=total, unit='B', unit_scale=True, desc="Downloading"
) as bar:
    for chunk in response.iter_content(chunk_size=8192):
        if not chunk:
            break
        f.write(chunk)
        bar.update(len(chunk))

Note: you may need to restart the kernel to use updated packages.


Downloading:   0%|          | 0.00/6.69G [00:00<?, ?B/s]

In [4]:
from zipfile import ZipFile
from tqdm.auto import tqdm
import os

out_dir  = "competition_support_set"

os.makedirs(out_dir, exist_ok=True)

with ZipFile(output_path, 'r') as z:
    for member in tqdm(z.infolist(), desc="Unzipping", unit="file"):
        z.extract(member, out_dir)

Unzipping:   0%|          | 0/10 [00:00<?, ?file/s]

# Set Weights and Biases Entity for tracking

In [8]:
# @title Set Weights and Biases Entity for tracking
entity = "arcinstitute" # @param {"type":"string","placeholder":"arcinstitute"}
! sed -i 's|entity: your_entity_name|entity: ${entity}|g' src/state/configs/wandb/default.yaml

# Training

In [3]:
# State TX Training Command
# This setups up training for State across datasets, using ESM2 featurizations
# of genes as the perturbation embeddings. Note that we are now generalizing
# across both contexts and perturbations (not just contexts)
! state tx train \
  data.kwargs.toml_config_path="competition_support_set/starter.toml" \
  data.kwargs.num_workers=4 \
  data.kwargs.batch_col="batch_var" \
  data.kwargs.pert_col="target_gene" \
  data.kwargs.cell_type_key="cell_type" \
  data.kwargs.control_pert="non-targeting" \
  data.kwargs.perturbation_features_file="competition_support_set/ESM2_pert_features.pt" \
  training.max_steps=400 \
  training.ckpt_every_n_steps=200 \
  model=state_sm \
  wandb.entity="arcinstitute" \
  wandb.project="state-training" \
  output_dir="competition" \
  name="first_run"

Traceback (most recent call last):
  File "/usr/local/bin/state", line 5, in <module>
    from state.__main__ import main
ModuleNotFoundError: No module named 'state'


In [6]:
# install wandb
# ! pip install wandb
import wandb

In [13]:
! state tx train --cfg job --help

usage: state tx train [-h] [hydra_overrides ...]

positional arguments:
  hydra_overrides  Hydra configuration overrides (e.g., data.batch_size=32)

options:
  -h, --help       show this help message and exit


In [21]:
# to make sure the state configue files wandb exist, reinstall the sate
# solve error hydra.errors.MissingConfigException: In 'config': Could not find 'wandb/default'
! uv run state --help --link-mode=copy

usage: state [-h] {emb,tx} ...

positional arguments:
  {emb,tx}

options:
  -h, --help  show this help message and exit


In [7]:
! ls file:///root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/configs

ls: cannot access 'file:///root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/configs': No such file or directory


In [10]:
! pwd

/workspace/state


# Run inference on the competition validation perturbations.

In [20]:
# View the available checkpoints
# This will be populated as you run training

! ls competition/first_run/checkpoints/

ls: cannot access 'competition/first_run/checkpoints/': No such file or directory


In [19]:
! state tx infer \
  --output "competition/prediction.h5ad" \
  --model_dir "competition/first_run" \
  --checkpoint "competition/first_run/checkpoints/final.ckpt" \
  --adata "competition_support_set/competition_val_template.h5ad" \
  --pert_col "target_gene"

Traceback (most recent call last):
  File "/root/.local/bin/state", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/__main__.py", line 68, in main
    run_tx_infer(args)
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/_cli/_tx/_infer.py", line 58, in run_tx_infer
    cfg = load_config(config_path)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/share/uv/tools/arc-state/lib/python3.11/site-packages/state/_cli/_tx/_infer.py", line 51, in load_config
    raise FileNotFoundError(f"Could not find config file: {cfg_path}")
FileNotFoundError: Could not find config file: competition/first_run/config.yaml


# Run Cell-Eval on the resulting anndata and submit your entry to the leaderboard.

In [ ]:
# install zstd for cell eval prep
! sudo apt install -y zstd

In [ ]:
! tool run --from git+https://github.com/ArcInstitute/cell-eval@main cell-eval prep -i competition/prediction.h5ad -g competition_support_set/gene_names.csv